In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [2]:
x = 0

# Functions required

In [4]:
def confidence_radii(t, delta):
    return math.sqrt(math.log(math.log(t)/delta)/t)

In [6]:
def intersection(lst1, lst2):
    temp = set(lst2)
    lst3 = [value for value in lst1 if value in temp]
    return lst3

In [7]:
def difference(li1, li2):
    s = set(li2)
    temp3 = [x for x in li1 if x not in s]
    return temp3

# Oracle

In [5]:
class Oracle:
    def __init__(self, M, N, mu_th) -> None:
        self.M = M
        self.N = N
        arms = 5
        attributes = 2
        self.mu_ij = [[0 for i in range (attributes)] for j in range (arms)]
        self.ab_array = [[[0, 0] for i in range (attributes)] for j in range (arms)]
        self.mu_i = [0 for i in range (arms)]

        self.mu_th = mu_th
        
        ## fixing alpha at 1

        self.ab_array[0] = np.array([[1, (1/x) - 1], [1, 0.01]])        #0.75 to 0.85
        self.ab_array[1] = np.array([[1, 1.5], [1, 1.5]])            #0.4
        self.ab_array[2] = np.array([[1, 4], [1, 1.5]])              #0.3
        self.ab_array[3] = np.array([[1, 9], [1, 1]])                #0.3
        self.ab_array[4] = np.array([[1, 1], [1, 2.33]])             #0.4

        for i in range (arms):
            for j in range (attributes):
                alpha = self.ab_array[i][j][0]
                beta = self.ab_array[i][j][1]
                self.mu_ij[i][j] = alpha / (alpha + beta)


        for i in range (len(self.mu_i)):
            self.mu_i[i] = sum(self.mu_ij[i])/attributes

    def play(self, arm):
        reward = np.zeros(self.M)
        for j in range(len(reward)):
            alpha, beta = self.ab_array[arm][j]
            reward[j] = np.random.beta(alpha, beta, 1)[0]
        return reward
    
    # def H_index(self):
    #     self.i_star = 0                 ## best arm in the feasible set
    #     self.i_star_star = 1            ## best arm in suboptimal set

    #     self.delta_i = [0 for i in range (self.N)]
    #     self.delta_ij = [[0 for i in range (self.M)] for j in range (self.N)]
    #     self.delta_i_attr = [0 for i in range (self.N)]

    #      ### assume that always i_star is 0 ###
        
    #     all_arms = list(range(0, self.N))

    #     self.perfect_feasible = get_perfect_feasible_set(self.mu_th, self.mu_ij)
    #     self.i_star = best_arm(self.mu_i, self.perfect_feasible)

    #     self.suboptimal = get_suboptimal_arm(self.mu_i[self.i_star], self.mu_i, all_arms)
    #     self.i_star_star = best_arm(self.mu_i, self.suboptimal)
        
    #     self.risky = difference(all_arms, self.suboptimal)
    #     self.infeasible = difference(all_arms, self.perfect_feasible) 

    #     for i in range (self.N):
    #         if (i != self.i_star):
    #             self.delta_i[i] = self.mu_i[self.i_star] - self.mu_i[i]
    #             self.delta_i_attr[i] = abs(min(self.mu_ij[i]) - self.mu_th)

    #     self.delta_i[self.i_star] = self.delta_i[self.i_star_star]
    #     self.delta_i_attr[self.i_star] = abs(min(self.mu_ij[self.i_star]) - self.mu_th)
    #     # print(self.delta_i_attr)

    #     self.set1 = intersection(self.perfect_feasible, self.suboptimal)
    #     self.set2 = intersection(self.infeasible, self.risky)
    #     self.set3 = intersection(self.infeasible, self.suboptimal)

    #     h0_den = (min(self.delta_i[self.i_star]/2, self.delta_i_attr[self.i_star]))**2
    #     h0 = 1/h0_den

    #     h1 = 0
    #     for i in self.set1:
    #         h1 += 4/(self.delta_i[i])**2

    #     h2 = 0
    #     for i in self.set2:
    #         h2 += 1/(self.delta_i_attr[i])**2

    #     h3 = 0
    #     for i in self.set3:
    #         h3 += 1/(max(self.delta_i[i]/2, self.delta_i_attr[i]))**2
    #         # h3 += 5*np.exp((1/(max(self.delta_i[i]/2, 2*self.delta_i_attr[i]))**2)/192)

    #     h_index = h0 + h1 + h2 + h3
    #     # print(self.delta_i_attr[1])
    #     # print(h0, h1, h2, h3)
    #     # print(self.set2)

    #     return h_index

        

# Main Function

In [ ]:
def game(M, N, mu_th, delta):
    T_t = [0 for i in range (N)]
    t = 1
    theGame = Oracle(M, N, mu_th)

    h_idx, h_idx_old = 0, 0
    # h_idx_old = theGame.H_index_old()
    h_idx = theGame.H_index()

    mu_hat_i = [0 for i in range (N)]
    mu_hat_ij = [[0 for i in range (M)] for j in range (N)]

    LCB = [0 for i in range (N)]
    UCB = [0 for i in range (N)]
    alpha = [0 for i in range(N)]

    LCB_attr = [[0 for i in range (M)] for j in range (N)]
    UCB_attr = [[0 for i in range (M)] for j in range (N)]

    T_unif = 20

    for j in range (T_unif):
        ## 20 times uniform exploration
        for i in range (N):
            reward_arm = theGame.play(i)
            for k in range (M):
                mu_hat_ij[i][k] = (mu_hat_ij[i][k] * T_t[i] + reward_arm[k])/(T_t[i] + 1)
            mu_hat_i[i] = sum(mu_hat_ij[i])/M
            T_t[i] += 1

    all_arms = list(range(0, N))
    feasible = list(range(0, N))
    t += N*T_unif